# Assignment 2

As before, if a question can be answered with 'yes/no', or a numeric value, you may simply state as much. If you incorporate code from the internet (which is not required and generally not advisable), please cite the source within your code (providing a URL is sufficient).

We will go through comparable code and concepts in the live learning sessions. If you run into trouble, start by using the help `help()` function in Python, to get information about the datasets and function in question. The internet is also a great resource when coding (though note that no outside searches are required by the assignment!). If you do incorporate code from the internet, please cite the source within your code (providing a URL is sufficient).

Please bring questions that you cannot work out on your own to office hours, work periods or share with your peers on Slack. We will work with you through the issue.

If you like, you may collaborate with others in the cohort. If you choose to do so, please indicate with whom you have worked with in your pull request by tagging their GitHub username. Separate submissions are required.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm

# Import specific objects
from sklearn.preprocessing import StandardScaler
from ISLP import load_data

### Question 1: Classification using KNN

We'll now use the `Caravan` dataset from the `ISLP` package. (You may use `Caravan.describe()` to review details of the dataset.) In this dataset, the response variable of interest is `Purchase`, which indicates if a given customer purchased a caravan insurance policy. We will simultaneously use all other variables in the dataset to predict the response variable.

In [5]:
# Load the "Caravan" dataset using the "load_data" function from the ISLP package
Caravan = load_data('Caravan')

# Add your code here
Caravan = Caravan.describe()
print(Caravan)

Before fitting any model, it is essential to understand our data. Answer the following questions about the `Caravan` dataset (Hint: use `print` and `describe`):  
_(i)_ How many observations (rows) does the dataset contain?    
_(ii)_ How many variables (columns) does the dataset contain?    
_(iii)_ What 'variable' type is the response variable `Purchase` (e.g., 'character', 'factor', 'numeric', etc)? What are the 'levels' of the variable?    
_(iv)_ How many predictor variables do we have (Hint: all variables other than `Purchase`)?  

In [6]:
# Add your code here
# (i) How many observations (rows) does the dataset contain?
num_rows = Caravan.shape[0]
print(f"Number of observations (rows): {num_rows}")

# (ii) How many variables (columns) does the dataset contain?
num_columns = Caravan.shape[1]
print(f"Number of variables (columns): {num_columns}")

# (iii) What 'variable' type is the response variable `Purchase`? What are the 'levels' of the variable?
purchase_type = Caravan['Purchase'].dtype
purchase_levels = Caravan['Purchase'].unique()
print(f"Variable type of 'Purchase': {purchase_type}")
print(f"Levels of 'Purchase': {purchase_levels}")

# (iv) How many predictor variables do we have?
num_predictor_vars = num_columns - 1  # Subtracting 1 for the 'Purchase' column
print(f"Number of predictor variables: {num_predictor_vars}")

# Optional: Displaying the first few rows to understand the data better
print(Caravan.head())

Number of observations (rows): 5822
Number of variables (columns): 86
Variable type of 'Purchase': object
Levels of 'Purchase': ['No' 'Yes']
Number of predictor variables: 85
   MOSTYPE  MAANTHUI  MGEMOMV  MGEMLEEF  MOSHOOFD  MGODRK  MGODPR  MGODOV  \
0       33         1        3         2         8       0       5       1   
1       37         1        2         2         8       1       4       1   
2       37         1        2         2         8       0       4       2   
3        9         1        3         3         3       2       3       2   
4       40         1        4         2        10       1       4       1   

   MGODGE  MRELGE  ...  APERSONG  AGEZONG  AWAOREG  ABRAND  AZEILPL  APLEZIER  \
0       3       7  ...         0        0        0       1        0         0   
1       4       6  ...         0        0        0       1        0         0   
2       4       3  ...         0        0        0       1        0         0   
3       4       5  ...         0      

Next, we must preform 'pre-processing' or 'data munging', to prepare our data for classification/prediction. For KNN, there are three essential steps. A first essential step is to 'standardize' the predictor variables. We can achieve this using the `scaler` method, provided as follows:

In [7]:
# Select predictors (excluding the 86th column)
predictors = Caravan.iloc[:, :-1]

# Standardize the predictors
scaler = StandardScaler()
predictors_standardized = pd.DataFrame(scaler.fit_transform(predictors), columns=predictors.columns)

# Display the head of the standardized predictors
print(predictors_standardized.head())

    MOSTYPE  MAANTHUI   MGEMOMV  MGEMLEEF  MOSHOOFD    MGODRK    MGODPR  \
0  0.680906  -0.27258  0.406697 -1.216964  0.779405 -0.694311  0.217444   
1  0.992297  -0.27258 -0.859500 -1.216964  0.779405  0.302552 -0.365410   
2  0.992297  -0.27258 -0.859500 -1.216964  0.779405 -0.694311 -0.365410   
3 -1.187437  -0.27258  0.406697  0.010755 -0.970980  1.299414 -0.948264   
4  1.225840  -0.27258  1.672893 -1.216964  1.479559  0.302552 -0.365410   

     MGODOV    MGODGE    MRELGE  ...   ALEVEN  APERSONG   AGEZONG  AWAOREG  \
0 -0.068711 -0.161816  0.427670  ... -0.20291 -0.073165 -0.081055 -0.05992   
1 -0.068711  0.464159 -0.096077  ... -0.20291 -0.073165 -0.081055 -0.05992   
2  0.914172  0.464159 -1.667319  ... -0.20291 -0.073165 -0.081055 -0.05992   
3  0.914172  0.464159 -0.619824  ... -0.20291 -0.073165 -0.081055 -0.05992   
4 -0.068711  0.464159  0.427670  ... -0.20291 -0.073165 -0.081055 -0.05992   

     ABRAND   AZEILPL  APLEZIER   AFIETS   AINBOED  ABYSTAND  
0  0.764971 -0.02

_(v)_ Why is it important to standardize the predictor variables?  
_(vi)_ Why did we elect not to standard our response variable `Purchase`?  


# Your answer here
'''Answer (v)
Standardizing the predictor variables is crucial for the following reasons:

Scale Invariance: Different predictor variables can have different units and scales (e.g., age in years, salary in dollars). Standardization ensures that each variable contributes equally to the distance calculations in KNN, preventing variables with larger scales from dominating the model.
Improved Model Performance: Many machine learning algorithms, including KNN, rely on distance metrics (like Euclidean distance). Standardizing ensures that the distance calculations are not biased towards predictors with larger numerical ranges.
Normalization: It brings all predictor variables to a common scale (usually with mean 0 and standard deviation 1), making the algorithm more sensitive to patterns in the data rather than the magnitude of the values.

Answer (vi)
The response variable Purchase is categorical, typically indicating whether a purchase was made ('Yes') or not ('No'). We do not standardize categorical variables because:

Nature of Data: Standardization is a process applied to numerical data. Since Purchase is categorical, standardizing it does not make sense. The concept of mean and standard deviation does not apply to categorical data.
Preserving Categories: We need to maintain the original categories ('Yes' and 'No') for the classification task. Transforming these categories would lose the meaning and context required for the model to learn and make predictions accurately.


_(vii)_ A second essential step is to set a random seed. Do so below (Hint: use the `random.seed` function). Why is setting a seed important? Is the particular seed value important? Why or why not?

Why is setting a seed important?

1. Reproducibility: Setting a random seed ensures that your code produces the same results each time it is run. This is crucial for verifying results, debugging, and sharing your work with others.
2. Consistency: It allows for consistent results across different runs of the same analysis or model, which is important for comparing performance metrics or debugging.

3. Collaborative Work: When sharing code with colleagues or publishing results, setting a seed ensures that others can replicate your exact results, which is a fundamental principle in scientific research.

Is the particular seed value important? Why or why not?

1. Choice of Seed Value: The specific value of the seed is generally not important. Any integer value can be used as a seed. The critical aspect is that the seed is set so that the random number generator produces the same sequence of numbers.

2. Fixed Sequence: Different seed values will produce different sequences of random numbers. However, once a seed is chosen and set, the sequence will be fixed and reproducible. Therefore, the specific value is less important than the act of setting it.

3. Documentation: It is good practice to document the seed value used in your experiments to ensure that others can reproduce your exact conditions if needed.

By setting a random seed, you ensure that the random processes involved in your analysis (e.g., splitting data into training and testing sets, initializing random weights in models) yield consistent and reproducible results.

In [8]:
# Add your code here
# (vii) Set a random seed
np.random.seed(42)

_(viii)_ A third essential step is to split our standardized data into separate training and testing sets. We will split into 75% training and 25% testing. The provided code randomly partitions our data, and creates linked training sets for the predictors and response variables. Extend the code to create a non-overlapping test set for the predictors and response variables.

In [9]:
# Create a random vector of True and False values
split = np.random.choice([True, False], size=len(predictors_standardized), replace=True, p=[0.75, 0.25])

# Define the training set for X (predictors)
training_X = predictors_standardized[split]

# Define the training set for Y (response)
training_Y = Caravan.loc[split, 'Purchase']

# Define the testing set for X (predictors)
testing_X = predictors_standardized[~split]

# Define the testing set for Y (response)
testing_Y = Caravan.loc[~split, 'Purchase']


_(ix)_ We are finally set to fit the KNN model. In Python, we can use the `KNeighborsClassifier()` function. Fit the KNN with k=1. (You may review arguments to knn by typing `help(knn.fit)`). 

In [10]:
# Add your code here
from sklearn.neighbors import KNeighborsClassifier

# Initialize the KNN model with k=1
knn = KNeighborsClassifier(n_neighbors=1)

# Fit the KNN model using the training data
knn.fit(training_X, training_Y)

# Display the model to confirm it has been fitted
print(knn)


KNeighborsClassifier(n_neighbors=1)


Using your fit model, answer the following questions:   
_(x)_ What is the prediction accuracy? (Hint: use the `score` method, and compare your model to `testing_Y`)  
_(xi)_ What is the predictor error ? (Hint: compute it from the accuracy)

In [11]:
# prediction accuracy rate
accuracy = knn.score(testing_X, testing_Y)
print(f"Prediction Accuracy: {accuracy:.2f}")

Prediction Accuracy: 0.90


In [12]:
# prediction error rate
error = 1 - accuracy
print(f"Predictor Error: {error:.2f}")

Predictor Error: 0.10


_(xii)_ How does this prediction error/accuracy compare to what could be achieved via random guesses? To answer this, consider the percent of customers in the `Caravan` dataset who actually purchase insurance, computed below:

In [13]:
# Calculate the percentage of customers who purchase insurance
percentage_purchase = (Caravan['Purchase'].eq('Yes').sum() / Caravan['Purchase'].eq('No').sum()) * 100

print(percentage_purchase)

6.357325538911216


_(xiii)_ Fit a second KNN model, with $K=3$. Does this model perform better (i.e., have higher accuracy, compared to a random guess)?

In [15]:
# Your code here
# Calculate the percentage of customers who do not purchase insurance
percentage_no_purchase = 100 - percentage_purchase

# Accuracy of random guessing (always guessing the majority class)
random_guess_accuracy = percentage_no_purchase / 100

print(f"Accuracy of random guessing: {random_guess_accuracy:.2f}")

# Initialize the KNN model with k=3
knn_3 = KNeighborsClassifier(n_neighbors=3)

# Fit the KNN model using the training data
knn_3.fit(training_X, training_Y)

# Calculate the prediction accuracy
accuracy_3 = knn_3.score(testing_X, testing_Y)

print(f"Accuracy of KNN with k=3: {accuracy_3:.2f}")



Accuracy of random guessing: 0.94
Accuracy of KNN with k=3: 0.93


# Criteria

|Criteria            |Complete           |Incomplete          |
|--------------------|---------------|--------------|
|Classification using KNN|All steps are done correctly and the answers are correct.|At least one step is done incorrectly leading to a wrong answer.|

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-2`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_2.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/applied_statistical_concepts/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ X ] Created a branch with the correct naming convention.
- [ X ] Ensured that the repository is public.
- [ X ] Reviewed the PR description guidelines and adhered to them.
- [ X ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.
